In [ ]:
import requests
import os
import urllib
import json
import re
import string
import xmltodict
from tqdm.auto import tqdm
import geopandas as gpd
from pandas import Series
import xml.etree.ElementTree as ET
from shapely.geometry import box
import math

In [2]:
import logging
logging.basicConfig(level=logging.INFO)
gpd.pd.options.display.max_colwidth = 300

In [3]:
connectId = "228b427a-fded-4f00-9602-1225879e86a6"
#aoi_file = './5_sites.geojson'
aoi_file = './sw_cement_annotations.geojson'

wfs_path = 'https://securewatch.maxar.com/catalogservice/wfsaccess'
xml_ns = {
    'dg': 'http://www.digitalglobe.com',
    'wfs': 'http://www.opengis.net/wfs',
    'gml': 'http://www.opengis.net/gml'
}


In [4]:
def xml_key(prefix, name):
    return "{" + xml_ns[prefix] + "}" + name

In [5]:
from pathlib import Path
Path('./metadata').mkdir(exist_ok=True)
Path('./images').mkdir(exist_ok=True)

# Buffer Geometry if Needed

In [6]:
%%time
df = gpd.read_file(aoi_file)

buf_radius_m = math.sqrt(2 * 1000 * 1000)/2

def box_buf(g):
    return box(*g.buffer(buf_radius_m).bounds).buffer(0)

g_types = df.geom_type.unique()
if len(g_types) == 1 and g_types[0] == 'Point':
    df['geometry'] = df.geometry \
        .to_crs(epsg=3395) \
        .apply(box_buf) \
        .to_crs(epsg=4326) 
    with open('./metadata/buffered.geojson', 'w') as f:
        f.write(df.to_json())

# Check areas    
#df.to_crs(epsg=3395).geometry.apply(lambda g: g.area)

CPU times: user 162 ms, sys: 11.4 ms, total: 173 ms
Wall time: 196 ms


# Construct Features DataFrame
This constructs a multiindex (`uid`, `featureId`) dataframe containing properties describing each available imagery asset for each AOI.

In [7]:
%%time
# NB, per Maxar:  minimum Y, minimum X, maximum Y, and maximum X
# cf: https://securewatchdocs.maxar.com/en-us/Miscellaneous/DevGuides/WFS/WFS_Feature.htm#BBOX
def mk_bounds(bounds) -> str:
    return f"{bounds[1]},{bounds[0]},{bounds[3]},{bounds[2]}"

def mk_query_payload(row: Series) -> dict:
    bbox = mk_bounds(row.geometry.bounds)
    payload = {
        "connectId": connectId,
        "Service": "WFS",
        "Request": "GetFeature",
        "version": "1.1.0",
        "srsName": "EPSG:4326",
        "typeName": "FinishedFeature",
        "BBOX": bbox,
        "WIDTH": 1,
        "HEIGHT": 1
    }
    return payload


def feature(row: Series):
    query = mk_query_payload(row)
    resp = requests.get(wfs_path, query)
    basename = row.uid
    try:
        with open(f'./images/{basename}-feature.xml', 'w') as f:
            f.write(resp.text)
        result = ET.fromstring(resp.text)
        rows = []
        for f in result.findall('.//dg:FinishedFeature', xml_ns):
            rows.append({
                **{re.sub('{.+}', '', c.tag): str(c.text) for c in f}, 
                **{'uid': basename, 'bbox': query["BBOX"]}
            })
        result = rows
    except Exception as e:
        result = str(e)
    return result

feature_set = df.apply(feature, axis=1)

all_features_df = gpd.pd.DataFrame(feature_set.explode().to_list()).set_index(['uid', 'featureId'])
all_features_df.to_excel('./metadata/all-features.xls')

CPU times: user 7.97 s, sys: 208 ms, total: 8.18 s
Wall time: 16.2 s


In [8]:
%%time

off_nadir_limit = 30.0
cloud_cover_limit = 0.4

def image_criteria(row):
    return ((row.productType == 'Pan Sharpened Natural Color') &
            (row.offNadirAngle.astype(float) < off_nadir_limit) &
            (row.cloudCover.astype(float) < cloud_cover_limit)
           )

viz_feature_df = all_features_df[image_criteria]

viz_feature_df.to_excel('./metadata/image-features.xls')

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


CPU times: user 2.62 s, sys: 11.9 ms, total: 2.63 s
Wall time: 2.69 s


# Construct Order

In [9]:
# This should be doable with multi-index slices, but couldn't figure out how 
rows = []
for uid, aoi_feature in viz_feature_df.groupby(by='uid', as_index=False):
    first = aoi_feature.sort_values(by='acquisitionDate', axis=0, ascending=False).reset_index().iloc[0]
    rows.append(first)
    
order_df = gpd.pd.DataFrame(rows)
order_df.to_csv('./order.csv')

print("off_nadir_limit|", "cloud_cover_limit|", "Order size")
print(off_nadir_limit, "|", cloud_cover_limit, "|", len(order_df))

off_nadir_limit| cloud_cover_limit| Order size
30.0 | 0.4 | 188


AOIs: 203

off_nadir_limit|cloud_cover_limit|Order size
---|---|---
25.0 | 0.3 | 181
30.0 | 0.4 | 188
25.0 | 0.5 | 184
35.0 | 0.5 | 188
15.0 | 0.4 | 131
35.0 | 0.6 | 188
45.0 | 0.8 | 188
30.0 | 0.4 | 188
